In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import math
import random as rand

df=pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx',header=None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [ ]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,0,1,2,3,4,5,6,7
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [ ]:
datan= df.values

x_d=datan[:,0:7]
y_d=datan[:,-1:]

In [ ]:
x_norm=np.ones((x_d.shape[0],x_d.shape[1]+1))
for i in range(x_d.shape[1]):
  x_norm[:,i+1:i+2]=(x_d[:,i:i+1]-np.mean(x_d[:,i:i+1]))/np.std(x_d[:,i:i+1])

In [ ]:
def train_test(x_norm,y_norm):
  x_train=x_norm[:math.floor(0.7*x_norm.shape[0])]
  x_val=x_norm[math.floor(0.7*x_norm.shape[0]):math.floor(0.8*x_norm.shape[0])]
  x_test=x_norm[math.floor(0.8*x_norm.shape[0]):]

  y_train=y_norm[:math.floor(0.7*y_norm.shape[0])]
  y_val=y_norm[math.floor(0.7*y_norm.shape[0]):math.floor(0.8*y_norm.shape[0])]
  y_test=y_norm[math.floor(0.8*y_norm.shape[0]):]
  return x_train,x_val,x_test,y_train,y_val,y_test

x_train,x_val,x_test,y_train,y_val,y_test= train_test(x_norm,y_d)

In [ ]:
def perfomance(x_tr,y_tr,w,b):
  pred=x_tr@w+b
  # print(pred)
  for i in range(len(pred)):
    if(pred[i]>0):
      pred[i]=1
    else:
      pred[i]=-1

  confusion_matrix=pd.crosstab(y_tr.flatten(),pred.flatten())
  confusion_matrix=np.asarray(confusion_matrix)
  acc=(confusion_matrix[0][0]+confusion_matrix[1][1])/len(x_te)*100
  print("Overall accuracy: ",acc)
  return pred,acc

In [ ]:
def SVM(X,Y,test,test_label,K,C,maxiter,val=None, val_label=None) :
  bi=0
  itr=0
  m=X.shape[0]
  E=np.zeros((m,1))
  u=np.zeros((m,1))
  bou=0.0001
  L=0
  H=0
  while (itr<maxiter):
      count=0
      for i in range(m):
          E[i] = np.sum((u*Y)*K[:,i]) - Y[i] + bi
          if ((u[i] < C and (Y[i] * E[i]) < -bou) or (u[i] > 0 and (Y[i] * E[i] > bou))):             
              j = int(rand.uniform(0,m))
              while j == i:
                  j =(int(rand.uniform(0,m))) #for j != i
              
              E[j] = bi + np.sum((u*Y)*K[:,j].reshape(-1,1)) - Y[j]
              u_i_old = u[i]
              u_j_old = u[j]
              if (Y[i] == Y[j]):
                  L = max(0, u[i] + u[j] - C)
                  H = min(C, u[i] + u[j])
              else:
                  L = max(0, -u[i] + u[j])
                  H = min(C, C - u[i] + u[j])

              if (L == H):
                  continue

              eta = 2 * K[i, j] - K[i, i] - K[j, j]
              if (eta >= 0):
                  continue
              
              u[j] = u[j] - (Y[j] * (E[i] - E[j])) / eta
              u[j] = min(H, u[j])
              u[j] = max(L, u[j])
              if (abs(u[j] - u_j_old) < bou):
                  u[j] = u_j_old
                  continue
              u[i] = u[i] + (Y[i] * Y[j] * (-u[j] + u_j_old))
              b1 = bi - E[i] - Y[i] * (u[i] - u_i_old) * K[i, j] - Y[j] * (u[j] - u[i]) * K[i, j]
              b2 = bi - E[j] - Y[i] * (u[i] - u_i_old) * K[i, j] - Y[j] * (u[j] - u[i]) * K[j, j]
              if (u[i] > 0 and u[i] < C):
                  bi = b1
              elif (u[j] > 0 and u[j] < C):
                  bi = b2
              else:
                  bi = (b1 + b2) / 2

              count += 1
      if(count==0):
          itr+=1
      else:
          itr=0
  Xsvm=[]
  Ysvm=[]
  mus_=[]
  for i in range(u.shape[0]):
      if u[i]>0:
          Ysvm.append(Y[i])
          Xsvm.append(X[i,:])
          mus_.append(u[i])
  Xsvm=np.array(Xsvm)
  Ysvm=np.array(Ysvm)
  mus_=np.array(mus_)

  w=np.dot((mus_*Ysvm).T,Xsvm).T

  p=np.dot(test,w)+bi
  q=[]
  for i in range(p.shape[0]):
      if p[i]>0:
          q.append(1)
      else:
          q.append(0)

  t=0
  for i in range(test_label.shape[0]):
      if test_label[i]==q[i]:
          t+=1

  p=np.dot(X,w)+bi
  q=[]
  for i in range(p.shape[0]):
      if p[i]>0:
          q.append(1)
      else:
          q.append(-1)

  l=0
  for i in range(Y.shape[0]):
      if Y[i]==q[i]:
          l+=1

  if val is not None:
    p=np.dot(val,w)+bi
    q=[]
    for i in range(p.shape[0]):
        if p[i]>0:
            q.append(1)
        else:
            q.append(0)
    o=0
    for i in range(val_label.shape[0]):
        if val_label[i]==q[i]:
            o+=1       
    return t*100/test_label.shape[0],o*100/val_label.shape[0],l*100/X.shape[0]
  return t*100/test_label.shape[0],l*100/X.shape[0]

One vs All


In [ ]:
y_te_1=np.zeros((len(y_test),1))

for i in range(len(y_test)):
  if y_test[i]==1:
    y_te_1[i]=1
  else:
    y_te_1[i]=-1


y_tr_1=np.zeros((len(y_train),1))

for i in range(len(y_train)):
  if y_train[i]==1:
    y_tr_1[i]=1
  else:
    y_tr_1[i]=-1

In [ ]:
a,class_acc1=SVM(x_train,y_tr_1,x_test,y_te_1,K=np.dot(x_train,x_train.T),C=0.1,maxiter=200)

In [ ]:
print('individual accuracy 1: ',class_acc1)

individual accuracy 1:  61.904761904761905


In [ ]:
y_te_1=np.zeros((len(y_test),1))

for i in range(len(y_test)):
  if y_test[i]==2:
    y_te_1[i]=1
  else:
    y_te_1[i]=-1


y_tr_1=np.zeros((len(y_train),1))

for i in range(len(y_train)):
  if y_train[i]==2:
    y_tr_1[i]=1
  else:
    y_tr_1[i]=-1

In [ ]:
a,class_acc2=SVM(x_train,y_tr_1,x_test,y_te_1,K=np.dot(x_train,x_train.T),C=0.1,maxiter=200)

In [ ]:
print('individual accuracy 2: ',class_acc2)


individual accuracy 2:  95.91836734693878


In [ ]:
y_te_1=np.zeros((len(y_test),1))

for i in range(len(y_test)):
  if y_test[i]==3:
    y_te_1[i]=1
  else:
    y_te_1[i]=-1


y_tr_1=np.zeros((len(y_train),1))

for i in range(len(y_train)):
  if y_train[i]==3:
    y_tr_1[i]=1
  else:
    y_tr_1[i]=-1

In [ ]:
a,class_acc3=SVM(x_train,y_tr_1,x_test,y_te_1,K=np.dot(x_train,x_train.T),C=0.1,maxiter=200)

In [ ]:
print('individual accuracy 3: ',class_acc3)


individual accuracy 3:  75.51020408163265


One vs One

In [35]:
c = 1
for model_num in range(1,3):
    for model_num_2 in range(model_num+1,4):
      x_norm_now=np.ndarray((0,x_norm.shape[1]))
      y_norm_now=np.ndarray((0,1))
      for y_index in range(len(y_d)):
        if y_d[y_index][0]==float(model_num) or y_d[y_index][0]==model_num:
          y_norm_now=np.concatenate((y_norm_now,-1*np.ones((1,1))),axis=0)
          x_norm_now=np.concatenate((x_norm_now,x_norm[y_index].reshape((1,x_norm.shape[1]))),axis=0)
        elif y_d[y_index][0]==float(model_num_2) or y_d[y_index][0]==model_num_2:
          y_norm_now=np.concatenate((y_norm_now,np.ones((1,1))),axis=0)
          x_norm_now=np.concatenate((x_norm_now,x_norm[y_index].reshape((1,x_norm.shape[1]))),axis=0)

      x_train,x_va,x_te,y_tr,y_va,y_te=train_test(x_norm_now,y_norm_now)
      a,class_acc=SVM(x_train,y_tr,x_te,y_te,K=np.dot(x_train,x_train.T),C=0.1,maxiter=200)

      print('for class ', c, 'acc is ',class_acc)
      c = c + 1

for class  1 acc is  81.63265306122449
for class  2 acc is  61.224489795918366
for class  3 acc is  100.0
